# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [7]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

browser = webdriver.Chrome(executable_path='chromedriver\chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
keepGoing = True
last_height = browser.execute_script("return document.body.scrollHeight")

while keepGoing:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('到達頁面底端')
        keepGoing = False
    else:
        last_height = new_height
        print('網頁更新中...')
        keepGoing = True
    '''    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


力派遣公司接到了一份任務有點模糊的工作：幫亞馬遜公司（Amazon.com Inc.）聽打一些錄音檔案。這份「數據助理」的工作是把一些隨意的語音內容逐字逐句打進筆電，時薪12美元（約新台幣361元）。
這些音檔包括了人們在自己家裡的私密談話。
時間是在2014年秋，當時亞馬遜剛推出的Echo居家智慧裝置最熱門話題就是聲控的虛擬助理Alexa。Alexa可以報新聞，解答問題，甚至陪小孩做功課。不過史拉蒂絲也開始理解，這套神奇AI其實背後需要靠很多和她一樣的真人才能夠運作。
亞馬遜把雲端裡的聲音指令收集下來，但仍需要真人「數據助理」協助訓練和提升這套系統。一開始，她以為她說話者是收了錢自願提供發音模式給機器研究。但顯然不是。
她聽到的錄音內容有時很怪異。有些孤單的人對Alexa吐露自己的私密的慾望和恐懼，也有人對著Alexa猛烈拍打。隨著Alexa大受歡迎，史拉蒂絲聽打的內容也越來越豐富。有些她的同事聽到小孩子報出了自己家裡的地址和電話，有人要Alexa幫忙訂購成人玩具，也有人在聚餐時聊說她們的聊天內容會不會被亞馬遜給偷聽去。
「這些人不可能知道有人正在聽他們說話，」她說：「這沒得到他們事先同意。」2016年，她辭掉了工作。
從史拉蒂絲辭職之後，已經有大約1/4的美國人購買了包括Echo、Google Home和Apple HomePod這類的「智慧音箱」。市場上的激烈競爭也帶動亞馬遜的Alexa，蘋果的Siri，谷歌的Google Assistant，微軟的Cortana，以及臉書的類似服務更加深入到人們的生活中。麥克風如今內建在手機、手錶、電視、冰箱、汽車以及其他冠以「智慧」之名的裝置之中。有人預估到2023年全球智慧音箱的市場銷售量將達到110億美元，全世界大概會有74億個聲控裝置被人們使用。平均每人大概有一個。
根據科技公司的說法，這些裝置並不會把你說的每句話都建檔，它們的智慧錄音裝置只有使用者打開時才會啟動。不過它們賣的裝置裡有些是在廚房裡或臥室裡，而且麥克風永遠開著，可能無意間就錄下了我們無意和他人分享的聲音。
參考資料：Bloomberg Businessweek
----------------------------------------------------------------------

[4] 中橡斥資上億打造全台首座碳黑研發中心　「黑